# What is purpose of Double Dueling DQN?
* Regular DQN tends to overestimate Q-values of potential actions in a given state
* Once one specific action becomes overestimated, it’s more likely to be chosen in the next iteration making it very 
hard for the agent to explore the environment uniformly and find the right policy.
* We will use our primary network to select an action and a target network to generate a Q-value for that action.
*  In order to synchronize our networks, we are going to copy weights from the primary network to the target one every 
'n' training steps.


In [1]:
import tensorflow as tf
import gym
import numpy as np
import cv2

We will be using the pong model

In [2]:
ENV_NAME = 'PongDeterministic-v4' 

# Preprocessing
We will have to downscale the image and grayscale it as it makes the policy faster. 
<br>
You can use opencv or skimage or even inbuilt tensorflow methods


In [3]:
class ProcessFrame:
    """Convert to GrayScale and resize"""
    def __init__(self, rows = 84, cols = 84):
        """
        Args:
            :param rows : Height to be resized to  
            :param cols : Width to be resized to
        """
        self.rows = rows
        self.cols = cols
    def process_frame(self, frame):
        """
        Processes the frame passed to required using OpenCV
        :param frame : An Atari game-play frame of (210,160,3)
        :return:      A processed frame of (84,84,1)
        """
        gray = cv2.cvtColor(frame, cv2.COLOR_RGB2GRAY)
        preprocessed_frame = cv2.resize(gray, [self.rows,self.cols])
        return preprocessed_frame

# Making the dueling network
Refer to [Wang Et Al 2016](https://arxiv.org/abs/1511.06581) for more info. 


The architecture used :
* The first convolutional layer has   32    8x8 filters with stride 4
* The second convolutional layer has  64    4x4 filters with stride 2
* The third convolutional layer has   64    3x3 filters with stride 1
* The fourth convolutional layer has  1024  7x7 filters with stride 1

Instead of directly predicting a single Q-value for each action, the dueling architecture 
splits the final convolutional layer into two streams that represent the value and advantage 
functions that predict a state value V(s) that depends only on the state, and action advantages 
A(s,a) that depend on the state and the respective action.

An excerpt from the original paper :

> Intuitively, the dueling architecture can learn which states are (or are not) valuable, without 
having to learn the effect of each action for each state. This is particularly useful in states 
where its actions do not affect the environment in any relevant way. In the experiments, we 
demonstrate that the dueling architecture can more quickly identify the correct action during 
policy evaluation as redundant or similar actions are added to the learning problem.

The state value V(s) talks about is the state is favourable or not while the action advantage A(s,a)
says which is the favourable action if in that state.  

Now to combine the state and advantage values : 

> Next, we have to combine the value and advantage stream into $Q$-values $Q(s,a)$. This is done 
the following way :
\begin{equation}
Q(s,a) = V(s) + \left(A(s,a) - \frac 1{| \mathcal A |}\sum_{a'}A(s, a')\right)
\end{equation}


To give a sense of motion to the network, we stack 4 frames and pass to the network to train upon.

Let's now implement this in a class. 


In [4]:
class DQN:
    """Implementation of the brain"""
    def __init__(self, n_actions, learning_rate = 0.00025, state_length = 4):
        """
        Init all required variables for class
        :param n_actions      : Number of actions available in the environment 
        :param learning_rate  : The specific lr argument for the Adam optimizer
        :param state_length   : The number of frames which create a state
        """
        self.n_actions = n_actions
        self.learning_rate = learning_rate
        self.state_length = state_length
        self.input_shape = [None, 84, 84, self.state_length]
        self.input = tf.placeholder(shape=self.input_shape, dtype=tf.float32)
        
        self.input /= 255       # Normalisation to help the learning process  
        
        # Making the layers
        self.conv1 = tf.layers.conv2d(inputs=self.input,
                                      filters=32,
                                      kernel_size=[8,8],
                                      strides=4,
                                      kernel_initializer=tf.variance_scaling_initializer(scale=2),
                                      padding='valid',
                                      activation=tf.nn.relu,
                                      use_bias=False,
                                      name='conv1')
        self.conv2 = tf.layers.conv2d(inputs=self.conv1,
                                      filters=64,
                                      kernel_size=[4,4],
                                      strides=2,
                                      kernel_initializer=tf.variance_scaling_initializer(scale=2),
                                      padding='valid',
                                      activation=tf.nn.relu,
                                      use_bias=False,
                                      name='conv2')
        self.conv3 = tf.layers.conv2d(inputs=self.conv2,
                                      filters=64,
                                      kernel_size=[3,3],
                                      strides=1,
                                      kernel_initializer=tf.variance_scaling_initializer(scale=2),
                                      padding='valid',
                                      activation=tf.nn.relu,
                                      use_bias=False,
                                      name='conv3')
        self.conv4 = tf.layers.conv2d(inputs=self.conv3,
                                      filters=1024,
                                      kernel_size=[7,7],
                                      strides=1,
                                      kernel_initializer=tf.variance_scaling_initializer(scale=2),
                                      padding='valid',
                                      activation=tf.nn.relu,
                                      use_bias=False,
                                      name='conv4')
        
        # Making 2 Streams
        self.valuestream, self.advantagestream = tf.split(value=self.conv4,
                                                          num_or_size_splits=2,
                                                          axis=3)
        self.valuestream = tf.layers.flatten(self.valuestream)
        self.advantagestream = tf.layers.flatten(self.advantagestream)
        self.advantage = tf.layers.dense(inputs=self.advantagestream, 
                                         units=self.n_actions, 
                                         kernel_initializer=tf.variance_scaling_initializer(scale=2), 
                                         name="advantage")
        self.value = tf.layers.dense(inputs=self.valuestream, 
                                     units=1, 
                                     kernel_initializer=tf.variance_scaling_initializer(scale=2), 
                                     name='value')
        
        # Combining value and advantage into Q-values
        self.q_values = self.value + tf.subtract(self.advantage, 
                                                 tf.reduce_mean(self.advantage, 
                                                                axis=1, 
                                                                keepdims=True))
        self.best_action = tf.argmax(self.q_values, axis=1)
        
        # targetQ according to Bellman equation: 
        # Q = r + gamma*max Q'
        self.target_q = tf.placeholder(shape=[None], dtype=tf.float32)
        # Action that was performed
        self.action = tf.placeholder(shape=[None], dtype=tf.int32)
        # Q value of the action that was performed
        self.Q = tf.reduce_sum(tf.multiply(self.q_values, 
                                           tf.one_hot(self.action, 
                                                      depth=self.n_actions, 
                                                      dtype=tf.float32)), 
                               axis=1)
        
        # Parameter updates
        self.loss = tf.reduce_mean(tf.losses.huber_loss(labels=self.target_q, predictions=self.Q))
        self.optimizer = tf.train.AdamOptimizer(learning_rate=self.learning_rate)
        self.update = self.optimizer.minimize(self.loss)

# Making the Get Action Class

For the first 50000 frames the agent only explores (ϵ=1). Over the following 1 million frames, ϵ is linearly
decreased to 0.1

After that we chose to decrease it to ϵ=0.01 over the remaining frames as suggested by the OpenAi Baselines for DQN.


In [5]:
class GetAction:
    """To implement the Epsilon Greedy Policy for returning actions"""
    def __init__(self, n_actions, eps_initial=1, eps_middle=0.1, eps_final=0.01, eps_eval=0,
                 train_start=50e3, train_middle=1e6, train_max=25e6):
        """
        Init all required variables for class
        :param n_actions     : Action Space for the environment
        :param eps_initial   : Start  Value for Epsilon
        :param eps_middle    : Middle Value for Epsilon
        :param eps_final     : Final  Value for Epsilon
        :param eps_eval      : Epsilon Value to be used during Evaluation
        :param train_start   : Number of frames to be pure exploration ie. Epsilon = 1
        :param train_middle  : Number of frames for epsilon to decay from eps_initial to eps_middle 
        :param train_max     : Number of frames for epsilon to decay from eps_middle to eps_final
        """
        self.n_actions = n_actions
        self.eps_initial = eps_initial
        self.eps_middle = eps_middle
        self.eps_final = eps_final
        self.eps_eval = eps_eval
        self.train_start = train_start
        self.train_middle = train_middle
        self.train_max = train_max
        
        # Now we need to generate linear lines for the decay between the start, middle, and the final points
        # m = slope = y2-y1 / x2-x1
        # c = intercept = y2 - m*x2
        self.m_1 = (self.eps_middle - self.eps_initial) / (self.train_middle - self.train_start)
        self.c_1 = self.eps_middle - self.m_1*self.train_middle
        self.m_2 = (self.eps_final - self.eps_final) / (self.train_max - self.train_middle)
        self.c_2 = self.eps_final - self.m_2*self.train_max
    
    def get_action(self, session, frame_number, state, dqn_object, evaluation=False):
        """
        Returns the action based on the epsilon greedy policy for the given state
        :param session       : The tensorflow session handling the prediction
        :param frame_number  : Number of frames passed. Used to determine if to use (m_1,c_1) or (m_2,c_2)
        :param state         : A stack of frames of the game-play after preprocessing ie. (84,84,4)
        :param dqn_object   : DDQN object to return the best action
        :param evaluation    : Flag to be set True while evaluation. Relies only upon the exploitation
        :return: An integer between 0 and n_actions-1 to be set as the action
        """
        if evaluation:
            eps = self.eps_eval
        elif frame_number < self.train_start:
            eps = self.eps_initial
        elif frame_number < self.train_middle:
            eps = self.m_1*frame_number + self.c_1
        elif frame_number < self.train_max:
            eps = self.m_2*frame_number + self.c_2
        
        if np.random.random() < eps:
            return np.random.randint(0, self.n_actions)
        else:
            return session.run(dqn_object.best_action, feed_dict={dqn_object.input:[state]})[0]  

# Experience Replay!!

> Second, learning directly from consecutive samples is inefficient, due to the strong correlations between 
the samples; randomizing the samples breaks these correlations and therefore reduces the variance of the updates. 
Third, when learning on-policy the current parameters determine the next data sample that the parameters are 
trained on. For example, if the maximizing action is to move left then the training samples will be dominated 
by samples from the left-hand side; if the maximizing action then switches to the right then the training 
distribution will also switch. It is easy to see how unwanted feedback loops may arise and the parameters 
could get stuck in a poor local minimum, or even diverge catastrophically. <br> [Mnih et al. 2013](https://arxiv.org/abs/1312.5602)

We add the experiences to a memory holding *deque*. The experiences are of the form - $(state,action,reward,next\_state,done)$.
We store the last 1 million experiences. We fill up the deque with our experiences and sample a batch of random 
experiences when needed to train. The deque is repopulated as more and more experiences are seen and older ones are
removed.

Lets make the class. We will sample from a memory buffer of 1 Million

In [6]:
class ExpMemory:
    """Storage for the experiences encountered . Stores the last 1 Million experiences"""
    def __init__(self, size=1e6, rows=84, cols=84, state_length=4, batch=32):
        """
        Init all required variables for class
        :param size          : The number of experiences to be stored  
        :param rows          : The pixel ht of a frame
        :param cols          : The pixel wd of a frame
        :param state_length  : The number of frames stacked to form a state
        :param batch         : The size of a batch to train upon
        """
        self.size = int(size)
        self.rows = rows
        self.cols = cols
        self.state_length = state_length
        self.batch = batch
        self.count = 0      # Holds the number of experiences in the deque
        self.current = 0    # Holds the index of the last updated experience
        # Memory of experiences made:
        self.actions = np.empty(self.size, dtype=np.int32)                             # Placeholder for actions
        self.rewards = np.empty(self.size, dtype=np.float32)                              # Placeholder for rewards                
        self.dones = np.empty(self.size, dtype=np.bool)                                 # Placeholder for done
        self.frames = np.empty((self.size,self.rows,self.cols), dtype=np.uint8)                         # Placeholder for frames
        self.states = np.empty((self.batch,self.state_length,self.rows,self.cols), dtype=np.uint8)       # Placeholder for stacked frames
        self.new_states = np.empty((self.batch,self.state_length,self.rows,self.cols), dtype=np.uint8)   # Placeholder for next stacked frames
        self.indices = np.empty(self.batch, dtype=np.int32)                               # Placeholder for sample of experience tuples
    def remember_state(self, action, frame, reward, done):
        """
        Adds the tuple of experience to the memory buffer
        :param action   : The action taken during the respective state 
        :param frame    : Preprocessed game-play frame (84,84,1)
        :param reward   : Reward associated with action
        :param done     : Done flag returned from taking the step
        :return: None
        """
        if frame.shape != (self.rows,self.cols):
            raise ValueError(f"Wrong dimensions of frame. Reqd - {self.rows},{self.cols} \t Passed - {frame.shape}")
        self.actions[self.current] = action             
        self.frames[self.current, ...] = frame
        self.rewards[self.current] = reward
        self.dones[self.current] = done
        self.current = (self.current+1) % self.size     # Implementation of DeQue on a list
        self.count = max(self.current,self.count)      
    def _get_state(self, index):
        """
        Gets the corresponding stacked state for the associated index 
        :param index : The index query  
        :return: A list of stacked frames of shape (4,84,84)
        """
        if self.count>0 and index>self.state_length-1:
            return self.frames[index-self.state_length+1 : index+1]
        else:
            raise ValueError("Minimum index of 3 is required. Populate the memory buffer")
    def _get_indices(self):
        """
        Gets a list of indices which are valid for experience replay training
        :return: A list of size (batch,)
        """
        for i in range(self.batch):
            while True:
                index = np.random.randint(low=self.state_length, high=self.count)   # Select a random index
                if index >= self.current >= index-self.state_length:                # State frames aren't continuous 
                    continue                                                        # ie. belong to two different times
                if self.dones[index-self.state_length : index].any():               # If any of the frames before were part of a different life
                    continue
                break
            self.indices[i] = index                                                 # Valid cases
    def get_experience(self):
        """
        Gets a batch of experience tuples of size self.batch
        :return: states, actions, rewards, next_state, dones  with self.batch=32 replays
        """
        if self.count<self.state_length:
            raise ValueError("Too few experiences to get s batch")
        self._get_indices()
        for i, index in enumerate(self.indices):
            self.states[i] = self._get_state(index)
            self.new_states[i] = self._get_state(index)
        # We now use transpose just to get the states in the form of (32,84,84,4) -> (batch,rows,cols,state_length)
        return np.transpose(self.states, axes=(0,2,3,1)), self.actions[self.indices], self.rewards[self.indices], \
               np.transpose(self.new_states, axes=(0,2,3,1)), self.rewards[self.indices]

# The Learning Part
The problem is that both $Qprediction$ and $Qtarget$ depend on the same parameters $θ$ if only one network is used. This 
can lead to instability when regressing $Qprediction$ towards $Qtarget$ because the "target is moving". We ensure a 
"fixed target" by introducing a second network with fixed and only occasionally updated parameters that estimates the 
target Q-values.
> Reinforcement learning is known to be unstable or even to diverge when a nonlinear function approximator such as a 
neural network is used to represent the action-value (also known as Q) function. This instability has several causes: 
the correlations present in the sequence of observations, the fact that small updates to Q may significantly change 
the policy and therefore change the data distribution, and the correlations between the action-values [...] and the 
target values [...]. We address these instabilities with a novel variant of Q-learning, which uses two key ideas. 
First, we used a biologically inspired mechanism termed experience replay that randomizes over the data, thereby 
removing correlations in the observation sequence and smoothing over changes in the data distribution [...]. 
Second, we used an iterative update that adjusts the action-values (Q) towards target values that are only 
periodically updated, thereby reducing correlations with the target.
[Mnih et al. 2015](https://www.nature.com/articles/nature14236/)

One network is used to predict the $Qprediction value while the other network is used to predict the $Qtarget$ value
and is fixed. The main network is updated by gradient descent while every 10000 steps the prediction network value is 
copied to the target network

In [7]:
def learn(sess, exp_memory, main_dqn, target_dqn, batch, gamma):
    """
    Implements the DQN equation
    :param sess         : A tensorflow session object
    :param exp_memory   : An object of ExpMemory
    :param main_dqn     : An object of DQN
    :param target_dqn   : An object of DQN
    :param batch        : A batch size to perform learning on
    :param gamma        : Discounter
    :return: None
    """
    states, actions, rewards, next_states, dones = exp_memory.get_experience()

    next_best_action = sess.run(main_dqn.best_action, feed_dict={main_dqn.input:next_states})   # Best action for next state from the main network
    q_vals = sess.run(target_dqn.q_values, feed_dict={target_dqn.input:next_states})            # Q Values for next state with target network
    double_q = q_vals[range(batch), next_best_action]

    target_q = rewards + (gamma*double_q * (1-dones))           # Bellman Equation
    # Gradient descend step to update the parameters of the main network
    loss, _ = sess.run([main_dqn.loss, main_dqn.update], 
                          feed_dict={main_dqn.input:states, 
                                     main_dqn.target_q:target_q, 
                                     main_dqn.action:actions})
    return loss

# Copy the networks
After every specific number of steps, it is required to match the target q values to the main q values.

We will now make a class for this.

In [8]:
class UpdateTargetNetwork:
    """Copies the weights of the main DQN to the target DQN"""
    def __init__(self, main_dqn, target_dqn):
        """
        Init all required variables for class
        :param main_dqn     : An object of DQN 
        :param target_dqn   : An object of DQN
        """
        self.main_dqn = main_dqn
        self.target_dqn = target_dqn
    def update_networks(self):
        """
        Copies the weights of the model-1 to model-2
        :return: None
        """
        self.target_dqn.set_weights(self.main_dqn.get_weights)

# Lets just Explore the world

In [9]:
env = gym.make(ENV_NAME)

In [10]:
env.unwrapped.get_action_meanings()

['NOOP', 'FIRE', 'RIGHT', 'LEFT', 'RIGHTFIRE', 'LEFTFIRE']

In [11]:
print(env.action_space, env.observation_space)

Discrete(6) Box(210, 160, 3)


In [12]:
# from time import sleep
# env.reset()
# for _ in range(1):
#     done = False
#     env.reset()
#     while not done:
#         obs, reward, done, info = env.step(env.action_space.sample())
#         env.render()
#         sleep(0.005)
#     env.close()


# Atari Player Class
A class which controls the movements is helpful. Lets make one

In [13]:
class Atari:
    """The player controller"""
    def __init__(self, env_name, state_length=4):
        """
        Init all required variables for class
        :param env_name     : Which environment do u want?
        :param state_length : The length of the stack of frames in a state
        """
        self.env = gym.make(env_name)
        self.state_length = state_length
        self.preprocess = ProcessFrame()
        self.state = None
    def step(self, action):
        """
        Performs an action and observes the reward and done status of it
        :param action   : Integer, the action to be done 
        :return: Experience tuple
        """
        next_frame, reward, done, info = self.env.step(action)
        processed_next_frame = self.preprocess.process_frame(next_frame)
        new_state = np.append(self.state[:,:,1:], processed_next_frame, axis=2)     # Creating the states
        self.state = new_state
        
        return processed_next_frame, reward, done, next_frame
    def reset(self):
        """
        Resets the environment and stacks four frames ontop of each other to create the first state
        :return: done status
        """
        frame = self.env.reset()
        self.last_lives = 0

        processed_frame = self.preprocess.process_frame(frame)
        self.state = np.repeat(processed_frame, self.state_length, axis=2)
        
        return done

# Getting all the Global variables
This includes the constants and the hyper-parameters

In [14]:
# Control parameters
import os
import time
MAX_EPISODE_LENGTH = 18000       # Equivalent of 5 minutes of gameplay at 60 frames per second
EVAL_FREQUENCY = 200000          # Number of frames the agent sees between evaluations
EVAL_STEPS = 10000               # Number of frames for one evaluation
NETW_UPDATE_FREQ = 10000         # Number of chosen actions between updating the target network. 
                                 # According to Mnih et al. 2015 this is measured in the number of 
                                 # parameter updates (every four actions), however, in the 
                                 # DeepMind code, it is clearly measured in the number
                                 # of actions the agent choses
DISCOUNT_FACTOR = 0.99           # gamma in the Bellman equation
REPLAY_MEMORY_START_SIZE = 50000 # Number of completely random actions, 
                                 # before the agent starts learning
MAX_FRAMES = 30000000            # Total number of frames the agent sees 
MEMORY_SIZE = 1000000            # Number of transitions stored in the replay memory
UPDATE_FREQ = 4                  # Every four actions a gradient descend step is performed
LEARNING_RATE = 0.00001          # The learning rate to be used for the gradient descent in Adam
BS = 32                          # Batch size

PATH = "output/"                 # checkpoints will be saved here
SUMMARIES = "summaries"          # logdir for tensorboard
RUNID = str(time.time())
os.makedirs(PATH, exist_ok=True)
os.makedirs(os.path.join(SUMMARIES, RUNID), exist_ok=True)
SUMM_WRITER = tf.summary.FileWriter(os.path.join(SUMMARIES, RUNID))

atari = Atari(ENV_NAME)

Making the two DQN - $Target$ and $Main$

In [15]:
with tf.variable_scope('mainDQN'):
    MAIN_DQN = DQN(atari.env.action_space.n, LEARNING_RATE)  
with tf.variable_scope('targetDQN'):
    TARGET_DQN = DQN(atari.env.action_space.n)       

init = tf.global_variables_initializer()
saver = tf.train.Saver()    

Instructions for updating:
Use keras.layers.conv2d instead.
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use keras.layers.flatten instead.
Instructions for updating:
Use keras.layers.dense instead.
Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Use tf.cast instead.


# TensorBoard
We will have the loss and rewards in the visualization.


In [16]:
# Scalar summaries for tensorboard: loss, average reward and evaluation score
with tf.name_scope('Performance'):
    LOSS_PH = tf.placeholder(tf.float32, shape=None, name='loss_summary')
    LOSS_SUMMARY = tf.summary.scalar('loss', LOSS_PH)
    REWARD_PH = tf.placeholder(tf.float32, shape=None, name='reward_summary')
    REWARD_SUMMARY = tf.summary.scalar('reward', REWARD_PH)
    EVAL_SCORE_PH = tf.placeholder(tf.float32, shape=None, name='evaluation_summary')
    EVAL_SCORE_SUMMARY = tf.summary.scalar('evaluation_score', EVAL_SCORE_PH)

PERFORMANCE_SUMMARIES = tf.summary.merge([LOSS_SUMMARY, REWARD_SUMMARY])

# Putting Everything Together
Lets call all the classes and implement the game agent

In [19]:
def train():
    """Contains the training and evaluation loops"""
    my_replay_memory = ExpMemory()
    network_updater = UpdateTargetNetwork(MAIN_DQN, TARGET_DQN)
    action_getter = GetAction(atari.env.action_space.n, train_max=MAX_FRAMES)

    with tf.Session() as sess:
        sess.run(init)
        frame_number = 0
        rewards = []
        loss_list = []
        
        while frame_number < MAX_FRAMES:
            
            ########################
            ####### Training #######
            ########################
            epoch_frame = 0
            while epoch_frame < EVAL_FREQUENCY:
                done = atari.reset()
                episode_reward_sum = 0
                for _ in range(MAX_EPISODE_LENGTH):
                    action = action_getter.get_action(sess, frame_number, atari.state, MAIN_DQN)   
                    processed_new_frame, reward, done, _ = atari.step(action)  
                    frame_number += 1
                    epoch_frame += 1
                    episode_reward_sum += reward
                
                    # Store transition in the replay memory
                    my_replay_memory.remember_state(action=action, 
                                                    frame=processed_new_frame[:, :, 0],
                                                    reward=reward, 
                                                    done=done)   
                    
                    if frame_number % UPDATE_FREQ == 0 and frame_number > REPLAY_MEMORY_START_SIZE:
                        loss = learn(sess, my_replay_memory, MAIN_DQN, TARGET_DQN, BS, gamma = DISCOUNT_FACTOR)
                        loss_list.append(loss)
                        
                    if frame_number % NETW_UPDATE_FREQ == 0 and frame_number > REPLAY_MEMORY_START_SIZE:
                        network_updater.update_networks()
                    
                    if done:
                        done = False
                        break

                rewards.append(episode_reward_sum)
                
                # Output the progress:
                if len(rewards) % 10 == 0:
                    # Scalar summaries for tensorboard
                    if frame_number > REPLAY_MEMORY_START_SIZE:
                        summ = sess.run(PERFORMANCE_SUMMARIES, 
                                        feed_dict={LOSS_PH:np.mean(loss_list), 
                                                   REWARD_PH:np.mean(rewards[-100:])})
                        
                        SUMM_WRITER.add_summary(summ, frame_number)
                        loss_list = []
                        
                    # Histogramm summaries for tensorboard
                    print(len(rewards), frame_number, np.mean(rewards[-100:]))
                    with open('rewards.dat', 'a') as reward_file:
                        print(len(rewards), frame_number, 
                              np.mean(rewards[-100:]), file=reward_file)
            
            ########################
            ###### Evaluation ######
            ########################
            terminal = True
            eval_rewards = []
            evaluate_frame_number = 0
            
            for _ in range(EVAL_STEPS):
                if done:
                    done = atari.reset(sess, evaluation=True)
                    episode_reward_sum = 0
                    done = False
               
                action_getter.get_action(sess, frame_number,
                                         atari.state, 
                                         MAIN_DQN,
                                         evaluation=True)
                processed_new_frame, reward, done, new_frame = atari.step(action)
                evaluate_frame_number += 1
                episode_reward_sum += reward
                
                if done:
                    eval_rewards.append(episode_reward_sum)
                     
            print("Evaluation score:\n", np.mean(eval_rewards))       
            
            #Save the network parameters
            saver.save(sess, PATH+'/my_model', global_step=frame_number)
            
            # Show the evaluation score in tensorboard
            summ = sess.run(EVAL_SCORE_SUMMARY, feed_dict={EVAL_SCORE_PH:np.mean(eval_rewards)})
            SUMM_WRITER.add_summary(summ, frame_number)
            with open('rewardsEval.dat', 'a') as eval_reward_file:
                print(frame_number, np.mean(eval_rewards), file=eval_reward_file)

In [21]:
train()

MemoryError: 